<a href="https://colab.research.google.com/github/vincent-grari/DIVERS/blob/main/Zonier_CM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys
print(sys.argv)
import os
import urllib
import os.path
import sklearn.preprocessing as preprocessing
from collections import namedtuple
import pandas as pd

['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-ac9be87b-ceb6-4c24-bd55-b485479fedf2.json']


In [2]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [3]:
!wget "http://grarivincent.com/research/baseINSEE.csv"

--2021-06-21 12:06:23--  http://grarivincent.com/research/baseINSEE.csv
Resolving grarivincent.com (grarivincent.com)... 217.160.0.183, 2001:8d8:100f:f000::2ce
Connecting to grarivincent.com (grarivincent.com)|217.160.0.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107089 (4.9M) [text/csv]
Saving to: ‘baseINSEE.csv’

baseINSEE.csv       100%[===================>]   4.87M  --.-KB/s    in 0.1s    

2021-06-21 12:06:23 (34.7 MB/s) - ‘baseINSEE.csv’ saved [5107089/5107089]



In [4]:
baseINSEE = (pd.read_csv("baseINSEE.csv", #names=column_names,
                    sep=r'\s*;\s*', engine='python', na_values=['NA']))

In [5]:
baseINSEE['pol_insee_code']= baseINSEE['code_commune_INSEE']

In [8]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
def load_ICU_Pricing(path):
    #column_names = ["CalYear","Gender","Type","Category","Occupation","Age","Group1","Bonus","Poldur","Value","Adind","SubGroup2","Group2","Density","Y"]
    #input_data = (pd.read_csv(path, #names=column_names,
    #                           sep=r',', engine='python', keep_default_na=False, na_values=['NA']))
    
    path = "http://grarivincent.com/research/db4ModelFull2.csv"
    input_data = (pd.read_csv(path, #names=column_names,
                                sep=r',', engine='python', na_values=['NA']))
    input_data=input_data.drop(columns=['Unnamed: 0'])
    
    base=baseINSEE
    base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
    base["pol_insee_code"] = base["pol_insee_code"].astype(int)
    base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

    input_data = input_data.drop_duplicates()
    input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
    input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

    input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
    #input_data = input_data[input_data['claim_amount']<=5000]
    input_data=input_data[input_data['claim_amount']!=0]
   
    #
    sensitive_attribs = ['Age']
    print(input_data.shape)
    G0 = (input_data.iloc[:, 37:180]) #137])
    #Z0 = pd.concat(Z0,input_data['long'])
    G0=G0.select_dtypes([np.number])
    G0 = G0.fillna(G0.mean())
    G=G0
    #Z = (Z0
    #    .drop(columns=['"REG"'
    #                   ])
    #    .fillna('Unknown').pipe(pd.get_dummies))

    y = input_data['claim_nb']
    E  = input_data['exposure']
    #I = input_data["pol_insee_code"]
    I = input_data[['pol_insee_code','latitude','longitude']]

    # features; note that the 'target' and sentive attribute columns are dropped
    X0 = input_data.iloc[:, 5:32]
    
    X = (X0
          .drop(columns=['pol_coverage','pol_insee_code'])
          .fillna('Unknown').pipe(pd.get_dummies))
    INSEE = input_data.iloc[:, 36]
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives E: {E.shape[0]} samples, {E.shape[0]} attributes")
    return X, y, G, E, I

X, y, G, E, I =load_ICU_Pricing("http://grarivincent.com/research/db4modelfull3.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(11025, 151)
features X: 11025 samples, 625 attributes
targets y: 11025 samples
sensitives E: 11025 samples, 11025 attributes


In [9]:
(y>0).mean()

1.0

In [10]:
G= G.iloc[:, :-5]

In [11]:
X_train, X_test, y_train, y_test, G_train, G_test, E_train, E_test, I_train, I_test  = train_test_split(X, y, G, E, I, test_size=0.2, random_state=7)

In [12]:
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler)

In [13]:
scalerg = StandardScaler().fit(G_train)
scale_df_g = lambda df, scaler: pd.DataFrame(scalerg.transform(df), columns=df.columns, index=df.index)
G_train = G_train.pipe(scale_df_g, scaler) 
G_test = G_test.pipe(scale_df_g, scaler)

In [14]:
y_traint =np.expand_dims(y_train,axis=1)
y_testt =np.expand_dims(y_test,axis=1)

### **GLM POISSON SANS ZONIER **

In [28]:
from sklearn import linear_model
reg = linear_model.GammaRegressor() 
reg.fit(X_train,y_train)

GammaRegressor()

In [39]:
def EDR_GAMMA(yhat, y):
  #loss=torch.mean(torch.exp(xbeta)-y*xbeta)
  #loss=torch.mean(yhat-y*torch.log(yhat))
  eps=0.000000000001
  res=1-np.mean(-np.log(y/yhat)+(y-yhat)/yhat)/np.mean(-np.log(y/np.mean(y))+(y-np.mean(y))/np.mean(y))
  return res

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [40]:
print(" GINI Poi without zonier via Reg : ", gini_coefficient(reg.predict(X_train)))
print(" GINI Poi without zonier via Reg : ", gini_coefficient(reg.predict(X_test)))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(reg.predict(X_train),y_train))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(reg.predict(X_test),y_test))
MSE_Poi_tr =  ((reg.predict(X_train)-y_train)**2).mean()
MSE_Poi_test =  ((reg.predict(X_test)-y_test)**2).mean()
print(" MSE Poi without zonier via Reg : ", MSE_Poi_tr)
print(" MSE Poi without zonier via Reg : ", MSE_Poi_test)

 GINI Poi without zonier via Reg :  0.020862612402880655
 GINI Poi without zonier via Reg :  0.020115198778913178
 EDR POIS Poi without zonier via Reg :  0.07828269788033726
 EDR POIS Poi without zonier via Reg :  -0.012907245669822265
 MSE Poi without zonier via Reg :  0.11435882930640887
 MSE Poi without zonier via Reg :  0.10882255645636843


In [44]:
reg.score(X_train,y_train)

0.07828269788052533

PRICING NN

In [45]:
!wget "https://raw.githubusercontent.com/vincent-grari/DIVERS/main/pricing_nn.py"

--2021-06-21 12:24:08--  https://raw.githubusercontent.com/vincent-grari/DIVERS/main/pricing_nn.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4768 (4.7K) [text/plain]
Saving to: ‘pricing_nn.py’

pricing_nn.py       100%[===================>]   4.66K  --.-KB/s    in 0s      

2021-06-21 12:24:08 (69.5 MB/s) - ‘pricing_nn.py’ saved [4768/4768]



In [46]:
from pricing_nn import Pricing_NN

In [97]:
class NN_G(nn.Module):
    def __init__(self):
        super(NN_G, self).__init__()
        self.fc1 = nn.Linear(G_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        h1 = torch.relu(self.fc1(x))
        #x = F.dropout(x, p=0.2)
        h2 = torch.relu(self.fc2(h1))
        #x = F.dropout(x, p=0.2)
        h3 = torch.relu(self.fc3(h2))
        #x = F.dropout(x, p=0.2)
        h4 = self.fc4(h3)
        return h4

class NN_GLM(nn.Module):    
    def __init__(self):
        super(NN_GLM, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 1)
    def forward(self, x,exposure):
        h1 = torch.exp(self.fc1(x))
        #h1 = self.fc1(x)+torch.log(exposure)
        return h1

In [105]:
P1_NN = Pricing_NN(regressor="gamma",mod_h=NN_GLM,mod_g= NN_G, batch_size = 128,nbepoch = 1024, lr = 0.0001, p_device= 'cuda')

In [106]:
from tqdm import tqdm 
from sklearn.utils import shuffle
from torch.autograd import Variable

In [107]:
P1_NN.fit(X_train, y_train, G_train, E_train)

DONE


In [112]:
P1_NN.predict(X_train, G_train, E_train)

tensor([[1.0618],
        [0.9148],
        [1.1551],
        ...,
        [1.0228],
        [1.1339],
        [0.9947]], device='cuda:0', grad_fn=<ExpBackward>)

In [113]:
P1_NN.predict_g(G_train)

tensor([[-0.0844],
        [ 0.0165],
        [-0.2853],
        ...,
        [-0.0461],
        [-0.2053],
        [-0.0933]], device='cuda:0', grad_fn=<AddmmBackward>)

In [114]:
y_pred2= P1_NN.predict(X_train, G_train, E_train).cpu().data.numpy()
y_pred2t= P1_NN.predict(X_test, G_test, E_test).cpu().data.numpy()


In [115]:
print(" GINI Poi without zonier via Reg : ", gini_coefficient(y_pred2))
print(" GINI Poi without zonier via Reg : ", gini_coefficient(y_pred2t))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(y_pred2,y_traint))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(y_pred2t,y_testt))
print(" MSE NN without zonier via Reg : ", np.mean((y_traint- y_pred2)**2))
print(" MSE NN without zonier via Reg : ", np.mean((y_testt- y_pred2t)**2))

 GINI Poi without zonier via Reg :  0.06887046569705275
 GINI Poi without zonier via Reg :  0.08683649230743354
 EDR POIS Poi without zonier via Reg :  0.4013960421238548
 EDR POIS Poi without zonier via Reg :  -0.3784471957742128
 MSE NN without zonier via Reg :  0.08023042489113691
 MSE NN without zonier via Reg :  0.1856795372648104
